**Задание**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [ ]:
# Импорт библиотеки Pandas:
import pandas as pd

In [ ]:
# Посмотрим на содержимое файла keywords.csv:
data = pd.read_csv('keywords.csv')
data.head(15)

In [ ]:
# Исходный словарь распределения городов по регионам:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Чтобы корректно классифицировать поисковые запросы по регионам, необходимо, чтобы гео-классификатор отпределял регион не только для названия города в именительном падеже, но и для склонений по остальным падежам.
Для данных целей подойдет модуль pymorphy2 (морфологический анализатор).
Ссылка на руководство пользователя модуля: https://pymorphy2.readthedocs.io/en/stable/user/guide.html

In [ ]:
# Установим модуль pymorphy2.
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# И импортируем его.
import pymorphy2

In [ ]:
# Для морфологического анализа используем класс MorphAnalyzer:
morph = pymorphy2.MorphAnalyzer()

С помощью модуля pymorphy2 преобразуем словарь geo_data, чтобы позже использовать его в гео-классификаторе (функция goroda()).

In [ ]:
# Преобразуем словарь geo_data: добавим в списки городов склонения названий данных городов.
# Для этого переберем каждую пару ключ-значение:
for key, val in geo_data.items():
# Создадим для каждого региона дополненый список городов, которым в словаре geo_data заменим первоначальный список:
  city_all = []
  geo_data[key] = city_all
  for city in val:
# Чтобы проскланять каждое название города в списке значений словаря geo_data, необходим объект Parse:
    city_nomn = morph.parse(city)[0]
# С помощью атрибута Parse.lexeme получим лексему слова (т.е. просклоняем его), для списка city_all отберем только первые 6 вариантов склонения (по падежам):
    for i in range(6):
      city_lex = city_nomn.lexeme[i][0]
      city_all.append(city_lex)

geo_data

{'Центр': ['москва',
  'москвы',
  'москве',
  'москву',
  'москвой',
  'москвою',
  'тула',
  'тулы',
  'туле',
  'тулу',
  'тулой',
  'тулою',
  'ярославль',
  'ярославля',
  'ярославлю',
  'ярославль',
  'ярославлем',
  'ярославле'],
 'Северо-Запад': ['петербург',
  'петербурга',
  'петербургу',
  'петербург',
  'петербургом',
  'петербурге',
  'псков',
  'пскова',
  'пскову',
  'псков',
  'псковом',
  'пскове',
  'мурманск',
  'мурманска',
  'мурманску',
  'мурманск',
  'мурманском',
  'мурманске'],
 'Дальний Восток': ['владивосток',
  'владивостока',
  'владивостоку',
  'владивосток',
  'владивостоком',
  'владивостоке',
  'сахалин',
  'сахалина',
  'сахалину',
  'сахалин',
  'сахалином',
  'сахалине',
  'хабаровск',
  'хабаровска',
  'хабаровску',
  'хабаровск',
  'хабаровском',
  'хабаровске']}

In [ ]:
# Напишем сам гео-классификатор:
def goroda(keyword):

# Соберем все города из словаря geo_data в отдельный список:
  cities = []
  for value in geo_data.values():
    for x in value:
      cities.append(x.lower())

# Проверим, содержатся ли слова из поискового запроса (столбец keyword файла keywords) в списке городов cities.
# Для этого преобразуем список городов и поисковый запрос в множества, найдем их пересечения.
# Если пересечение имеется, то добавим этот город (или города) в новый список пересечений inters:
  inters = []
  for word in keyword.split():
      for el in cities:
        if word == el:
          inters.append(word)

# Если список пересечений пустой, в новый столбец датафрейма запишем значение 'undefined'
  if inters == []:
    return 'undefined'
  else:
    for key, val in geo_data.items():
      for el in val:
# Если список пересечений не пустой, в новый столбец запишем название региона (значение ключа),
# соответствующее городу с индексом 0 из списка пересечений (на случай, если в список пересечений попадет несколько городов):
#if el == inters[0]:
        if str(inters[0][0:(len(inters)-3)]) in el:
          return key

In [ ]:
# С помощью инструментов библиотеки Pandas сформируем искомый датафрейм:
data['region'] = data['keyword'].apply(goroda)
data.head(1000)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
9995,курьер среда бердск,34421,undefined
9996,слушать песни 2016 года новинки русские слушат...,27721,undefined
9997,world of warcraft,27464,undefined
9998,дети74,27457,undefined


In [ ]:
# Сгруппируем строки по региону, подсчитаем общее количество запросов, отнесенных к каждому региону:
data.groupby('region').count().head()

,keyword,shows
region,,
undefined,98891,98891
Дальний Восток,85,85
Северо-Запад,238,238
Центр,786,786
